In [3]:
import pandas as pd

In [76]:
#reading CSV tables
admissions = pd.read_csv("./data/ADMISSIONS.csv")
patients = pd.read_csv("./data/PATIENTS.csv")
services = pd.read_csv("./data/SERVICES.csv")
icustays = pd.read_csv("./data/ICUSTAYS.csv")
drgcodes = pd.read_csv("./data/DRGCODES.csv")

In [77]:
#tables joins to data frame
df = admissions.join(patients.set_index("subject_id"), on = "subject_id", lsuffix="_adm", rsuffix="_pat")
df = df.join(services.set_index(["subject_id", "hadm_id"]), on = ["subject_id", "hadm_id"])
df = df.join(icustays.set_index(["subject_id", "hadm_id"]), on = ["subject_id", "hadm_id"], rsuffix="_icu")
df = df.join(drgcodes.set_index(["subject_id", "hadm_id"]), on = ["subject_id", "hadm_id"], rsuffix="_drg")

Kilka luźnych uwag do zanotowania:
- część zmiennych tworzy wiele rekordów do pojedynczego pobytu pacjenta (np. curr_service, te od kodów DRG), a df musi mieć formę: wiersz opisujący pobyt pacjenta - target 
- trzeba te zmienne zagregować albo z nich zrezygnować
- z tych powodów nie wziąłem np microbiology events
- zmiennych jest 15 więc spokojnie może coś odpaść przy EDA
- CSVki w folderze jak wyżej umieście i nie wrzucamy go na repo, jak będziemy mieli docelowy df to wtedy trzeba zapisać do CSV i na repo
- jak jakieś inne zmienne itp. to kodu wiele nie ma wiadomo co i gdzie możecie zmieniać

In [78]:
df = df[[
    #identyfikator pobytu pacjenta: subject_id + hadm_id
    "subject_id", "hadm_id",
    ###ADMISSIONS
    #czas przyjęcia, wypisu
    "admittime", "dischtime",
    #typ pobytu
    "admission_type",
    #dane personalne
    "insurance", "language", "religion", "marital_status", "ethnicity",
    #TARGET zmienna binarna śmierć w szpitalu
    "hospital_expire_flag",
    ###PATIENTS
    "gender", 
    #data urodzenia
    "dob",
    ###SERVICES
    #opieka/zabieg
    "curr_service",
    ###ICUSTAYS
    #długość pobytu na ICU
    "los",
    ###DRGCODES
    #para identyfikująca 
    "drg_type", "drg_code",
    "drg_severity", "drg_mortality",
]]

In [75]:
df

,subject_id,hadm_id,admittime,dischtime,admission_type,insurance,language,religion,marital_status,ethnicity,hospital_expire_flag,gender,dob,curr_service,los,drg_code
0,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,EMERGENCY,Medicare,NaN,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,0,F,2094-03-05 00:00:00,MED,1.6325,416
1,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,EMERGENCY,Private,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,1,F,2090-06-05 00:00:00,MED,13.8507,205
2,10013,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,EMERGENCY,Medicare,NaN,CATHOLIC,NaN,UNKNOWN/NOT SPECIFIED,1,F,2038-09-03 00:00:00,MED,2.6499,416
3,10017,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,EMERGENCY,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,0,F,2075-09-21 00:00:00,MED,2.1436,3153
3,10017,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,EMERGENCY,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,0,F,2075-09-21 00:00:00,MED,2.1436,491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,44222,192189,2180-07-19 06:55:00,2180-07-20 13:00:00,EMERGENCY,Medicare,ENGL,CATHOLIC,SINGLE,WHITE,0,M,2107-06-27 00:00:00,CMED,1.3279,2013
127,44222,192189,2180-07-19 06:55:00,2180-07-20 13:00:00,EMERGENCY,Medicare,ENGL,CATHOLIC,SINGLE,WHITE,0,M,2107-06-27 00:00:00,CMED,1.3279,308
128,44228,103379,2170-12-15 03:14:00,2170-12-24 18:00:00,EMERGENCY,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,0,F,2112-10-22 00:00:00,SURG,4.6191,7203
128,44228,103379,2170-12-15 03:14:00,2170-12-24 18:00:00,EMERGENCY,Private,ENGL,NOT SPECIFIED,SINGLE,WHITE,0,F,2112-10-22 00:00:00,SURG,4.6191,7203
